# Introduction

This project aims to use the seq2seq model to develop a calculator that does simple arithmetics, such as addition, without directly evaluating the mathematical expression. The seq2seq model is a machine translation model developed by Google, and it is the magic behind Google Translate. By using this model, we will demonstrate that it is possible to evaluate mathematical expressions as literal strings and, as a result, still give the correct output.

# Background

Before we jump into the seq2seq model, let's first take a look at the progression from basic neural networks, to recurrent neural networks, and eventually, to long short-term memory networks, which will be used in this project.

## Artificial Neural Networks (ANN)

What is a neural network? It's hard to explain it in just a couple of sentences, but put it in simpler terms, it essentially boils down to pattern recognition. For example, fitting a non-linear curve is something that can be done using a ANN. 

However, the original ANN was not that intelligent in a sense that information gets lost during the training process. Using the analogy from colah's blog, it's as if human starts thinking from scratch every second, so no previous information was retained. 

For example, imagine a 5 years old child trying to predict the next letter of a sequence like "ABCABCAB_". The original ANN is actually not able to solve this letter pattern problem since it fails to retain the information of the last letter seen, which a 5 yeras old may be able to do. 

## Recurrent Neural Networks (RNN)

The RNN addreses the problem mentioned before, that is the major shortcoming of ANN, that is it is not capable of retaining information from a previous state.

This problem is fixed by the idea of back propagation, which creates loops in the neural network, allowing information to persist in the network.

Using the analygy before of predicting the next letter from the sequence of "ABCABCAB_", this time, the neural network will be aware of the previous state, in this case, the last letter we know seems to be a "B", and thus, according to the "ABC" pattern, the neural network can arrive at a conclusion that the following letter would be a "C".

I would also like to dig a bit more into the neural network training process here and mention the idea of weights and bias. Each neurons in the neural networks has its own weights and bias parameters, and they are the diciding factors of what's the final predict is going to be. 

When training such a network, initially, the prediction may seem random, which is normal. But the RNN will use back propagation, essentially the cencept of gradient descent in calculus 3, to adjust the weights and biases of each neuron so that we can get closer and closer to the correct output. The weights and bias will learn how far off they are from the optimal result and adjust themselves towards that direction.



Let's take a look at another problem mentioned again in colah's blog.

For example, we may have a problem which asks us to predict the next word based on previous ones. Let's say we have "the clouds are in the ___" as the input, and we can see that it's pretty obvious that the next word is going to be "sky". In this case, RNN uses the recent information to perform the present task. In other word, RNN performs well when the distance between the context and place is small.

However, if we present with the input such as "I grew up in France… I speak fluent ______.", a RNN will struggle with this problem since it requires further context of France, which may be way back of the place of prediction. At in reality, it is very likely that we have big distance gaps between the relevant information and the prediction place.


## Long Short-Term Memory Networks (LSTM)

LSTM is a type of RNN that solves the problem mentioned above. As its name suggests, it is able to remember information for a long period of time, so it would not struggle to solve the France French problem.

seq2seq model is a popular application of LSTM networks, and it is often used in language translation problems, which is very context dependent.

In this project, instead of doing language translations, we will apply seq2seq to arithemetic. Just like language translations, the neural network does not actually understand the meaning of every sentences. In this case, our neural network has no idea about the concept of numbers or arithmetics. Therefore, it's like translating the mathmetical expressions to answers, which get inputed as string forms not numbers and outputed in string forms as well.

# Coding!


Where the fun begins :)

## Initial Setup

In [ ]:
# Expression Parameters
digit = 5
max_expr_len = digit*2+1
max_ans_len = digit+1

## Data Generation


Here we focus only on addition operations of two integers from 1-5 digits. Once that works, we may expand it to other operations such as subtraction, multiplication, division, etc.

Notice that given the nature of our problem, we don't need an existing data set since everything can be generated by ourselves.

In [ ]:
import numpy as np
np.random.seed(2333) # Set a seed

In [ ]:
# Generate a integer less than or equal to the number of digits defined above
gen_num = lambda: int("".join(np.random.choice(tuple('0123456789')) for _ in range(np.random.randint(1, digit+1))))

The code below is a bad idea because it's probability of getting numbers with different amounts of digits is not equal. It is more likely to get the most amount of digits possible.

In [ ]:
# gen_num = lambda: np.random.randint(0, 10**digit) # DO NOT do this!

Let's now populate our data set (expressions and answers).

In [ ]:
seen = set() # Tracking duplications

# Generate data for the addition operation
def gen_addition(size):
  expressions = []
  answers = []

  while len(expressions) < size:
    # a + b = c
    a, b = gen_num(), gen_num()
    c = a + b

    # Convert to string forms for training
    exp = f'{a}+{b}'
    ans = str(c)

    # Check for duplications
    key = tuple(sorted((a,b)))
    if key in seen:
      continue
    seen.add(key)

    # Populate lists and adjust according to seq2seq model
    # Making all expressions the same length, as well as all answers
    expressions.append(exp+' '*(max_expr_len-len(exp)))
    answers.append(' '*(max_ans_len-len(ans))+ans) 
  return expressions, answers

Example output:

In [ ]:
example_X, example_y = gen_addition(1)
example_X, example_y

(['3+666      '], ['   669'])

## One-hot Encoding and Decoding

The reason we need this step is because, we can't work with neural networks using characters, so we need to fisrt convert everything into numbers, and this is what one-hot encoding and decoding is for. By doing this, we transformed our problem into a classification problem, where the neural network will give predictions for, in this case, 11 different classes ('0123456789+ ').

The overall process would be, we need to first assign a id to each characters, then we encode all ids using one-hot encoding. The reverse process applies to one-hot decoding.

In [ ]:
chars = '0123456789+ ' # All the possibile charecters for inputs and outputs

def one_hot_encode(exp, chars):
  max_len = len(chars)

  # Dictionary for char to id encoding
  char2id = {c: i for i, c in enumerate(chars)} 

  # One-hot encoding for expression
  seq_encoded = list()
  for seq in exp:
    pattern = list()
    for i in seq:
      vector = np.zeros(max_len)
      vector[char2id[i]] = 1
      pattern.append(vector)
    seq_encoded.append(pattern)

  return np.array(seq_encoded)  

def one_hot_decode(seq, chars):
  # Dictionary for id to char decoding
  id2char = {i: c for i, c in enumerate(chars)}

  # Decode seq one by one back to chars, and join them to form answers
  strings = list()
  for pattern in seq:
    string = id2char[np.argmax((pattern))]
    strings.append(string)
  return ''.join(strings)

Example output:

In [ ]:
example_X_encoded, example_y_encoded = one_hot_encode(example_X, chars), one_hot_encode(example_y, chars)
example_X_encoded, example_y_encoded

(array([[[0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0.],
         [0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1.],
         [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1.],
         [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1.],
         [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1.],
         [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1.],
         [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1.]]]),
 array([[[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1.],
         [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1.],
         [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1.],
         [0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0.,

In [ ]:
example_X_decoded = "".join([one_hot_decode(seq, chars) for seq in example_X_encoded])
example_y_decoded = "".join([one_hot_decode(seq, chars) for seq in example_y_encoded])
example_X_decoded, example_y_decoded

('6697+13460 ', ' 20157')

## seq2seq LSTM Model Configuration

In [ ]:
from keras.models import Sequential
from keras.layers import LSTM, TimeDistributed, RepeatVector, Dense

After countless trials, it seems like having three LSTM layers gives us the optimal result.

In [ ]:
model = Sequential()
model.add(LSTM(128, input_shape=(max_expr_len, len(chars))))
model.add(RepeatVector(digit + 1))
model.add(LSTM(64, return_sequences=True))
model.add(LSTM(32, return_sequences=True))

model.add(TimeDistributed(Dense(len(chars), activation='softmax')))
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 128)               72192     
                                                                 
 repeat_vector (RepeatVector  (None, 6, 128)           0         
 )                                                               
                                                                 
 lstm_1 (LSTM)               (None, 6, 64)             49408     
                                                                 
 lstm_2 (LSTM)               (None, 6, 32)             12416     
                                                                 
 time_distributed (TimeDistr  (None, 6, 12)            396       
 ibuted)                                                         
                                                                 
Total params: 134,412
Trainable params: 134,412
Non-trai

## Train Test Split

In [ ]:
from sklearn.model_selection import train_test_split

X, y = gen_addition(100000) # Train the model on 100000 data
X, y = one_hot_encode(X, chars), one_hot_encode(y, chars) # Encode everything

# 60% training, 20% validading, 20% testing
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=2333)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.25, random_state=2333)

## Model Training

ATTENTION: Skip the cell below if you don't want to train the model on the spot, we will provide a pretrained model after.

Note that by having a validation set, we noticed a great degree of improvement in terms of accuracy.

In [ ]:
model.fit(X_train, y_train, epochs=64, validation_data=(X_val, y_val), batch_size=64 ,verbose=1)

Epoch 1/64
938/938 [==============================] - 41s 39ms/step - loss: 1.7216 - accuracy: 0.3512 - val_loss: 1.6189 - val_accuracy: 0.3712
Epoch 2/64
938/938 [==============================] - 35s 38ms/step - loss: 1.5366 - accuracy: 0.4016 - val_loss: 1.4734 - val_accuracy: 0.4266
Epoch 3/64
938/938 [==============================] - 35s 37ms/step - loss: 1.4341 - accuracy: 0.4432 - val_loss: 1.3852 - val_accuracy: 0.4684
Epoch 4/64
938/938 [==============================] - 36s 38ms/step - loss: 1.3516 - accuracy: 0.4779 - val_loss: 1.3260 - val_accuracy: 0.4845
Epoch 5/64
938/938 [==============================] - 35s 37ms/step - loss: 1.2937 - accuracy: 0.4958 - val_loss: 1.2626 - val_accuracy: 0.5094
Epoch 6/64
938/938 [==============================] - 35s 37ms/step - loss: 1.2472 - accuracy: 0.5142 - val_loss: 1.2160 - val_accuracy: 0.5258
Epoch 7/64
938/938 [==============================] - 35s 37ms/step - loss: 1.1968 - accuracy: 0.5335 - val_loss: 1.1613 - val_accuracy:

In [ ]:
# model.save('seq2seq-arithmetic.h5')

Training can take a long time, so here is a pre-trained model for quick access!

In [ ]:
!wget -O seq2seq-arithmetic.h5 'https://github.com/itsjoeoui/seq2seq-arithmetic/raw/main/seq2seq-arithmetic.h5'

--2022-04-17 00:18:05--  https://github.com/itsjoeoui/seq2seq-arithmetic/raw/main/seq2seq-arithmetic.h5
Resolving github.com (github.com)... 192.30.255.112
Connecting to github.com (github.com)|192.30.255.112|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/itsjoeoui/seq2seq-arithmetic/main/seq2seq-arithmetic.h5 [following]
--2022-04-17 00:18:05--  https://raw.githubusercontent.com/itsjoeoui/seq2seq-arithmetic/main/seq2seq-arithmetic.h5
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.110.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1670544 (1.6M) [application/octet-stream]
Saving to: ‘seq2seq-arithmetic.h5’

seq2seq-arithmetic. 100%[===================>]   1.59M  --.-KB/s    in 0.03s   

2022-04-17 00:18:06 (48.0 MB/s) - ‘seq2seq-arithmetic.h5

In [ ]:
import tensorflow as tf
model = tf.keras.models.load_model('seq2seq-arithmetic.h5')

## Model Evaluation

In [ ]:
result = model.predict(X_test, verbose=0) # Get all predictions

# Decode everything
expressions = [one_hot_decode(x, chars) for x in X_test]
predicted = [one_hot_decode(x, chars) for x in result]
expected = [one_hot_decode(x, chars) for x in y_test]

In [ ]:
correct = 0
for i in range(len(result)):
    if expected[i] == predicted[i]:
        correct += 1
    else:
        # Print all wrong predictions
        print(f"Expression: {expressions[i]}. Predicted: {predicted[i].ljust(max_ans_len)}. Expected: {expected[i].ljust(max_ans_len)}.")

Expression: 207+84     . Predicted:    281. Expected:    291.
Expression: 555+71980  . Predicted:  72635. Expected:  72535.
Expression: 9979+54139 . Predicted:  64108. Expected:  64118.
Expression: 8326+6648  . Predicted:  15964. Expected:  14974.
Expression: 7130+1574  . Predicted:   8604. Expected:   8704.
Expression: 4010+1     . Predicted:   4012. Expected:   4011.
Expression: 7+1152     . Predicted:   1169. Expected:   1159.
Expression: 86266+3    . Predicted:  86279. Expected:  86269.
Expression: 6+21095    . Predicted:  21001. Expected:  21101.
Expression: 5309+19327 . Predicted:  24626. Expected:  24636.
Expression: 629+85767  . Predicted:  86496. Expected:  86396.
Expression: 9027+87478 . Predicted:  97505. Expected:  96505.
Expression: 45380+42351. Predicted:  88721. Expected:  87731.
Expression: 7687+91815 . Predicted: 100502. Expected:  99502.
Expression: 73+54826   . Predicted:  54809. Expected:  54899.
Expression: 14218+9131 . Predicted:  24249. Expected:  23349.
Expressi

Notice that in all wrong predictions, we are usually only off by one digit $\pm$ 1. Not bad at all!

In [ ]:
print(f"Accuracy: {round((correct/len(result))*100, 2)}% based on our test set.")

Accuracy: 85.61% based on our test set.


# Playground

You will be asked to give two numbers less than and equal to 5 digits, and we will use our trained model to give a predictive output.

In [ ]:
while True:
  a = input("Enter the 1st positive integer (1-5 digits): ")
  b = input("Enter the 2nd positive integer (1-5 digits): ")

  if not a.isnumeric() or not b.isnumeric():
    print("Exit...")
    break

  expression = a + "+" + b
  answer = str(int(a)+int(b))

  expression = [expression+' '*(max_expr_len-len(expression))]
  expression_enc = one_hot_encode(expression, chars)
  result = model.predict(expression_enc, verbose=0) # Get prediction

  predicted = one_hot_decode(result[0], chars).lstrip()
  if predicted == answer:
    print("[CORRECT]", end=" ")
  else:
    print("[OOPS]", end=" ")
  print(f"Expression: {expression[0]}. Predicted: {predicted.ljust(max_ans_len)}. Expected: {answer.ljust(max_ans_len)}.")
  print("\n")

Enter the 1st positive integer (1-5 digits): 1234
Enter the 2nd positive integer (1-5 digits): 4321
[CORRECT] Expression: 1234+4321  . Predicted: 5555  . Expected: 5555  .


Enter the 1st positive integer (1-5 digits): 
Enter the 2nd positive integer (1-5 digits): 
Exit...


# Conclusion

It was definitely interesting to see how LSTM networks are able to do simple arithemic without any idea about what numbers are and what does the addition symbol supposed to mean. In the end, we were able to build a model with over 85% percent accuracy, which is already pretty decent given that most incorrect outputs are only off by one or two digits. 

A couple of interesting things we noticed during the process. 

1. With fewer LSTM layers, there is a great amount of decrese between the last layer and the one before it, jumping from around 40k neurons to only 396, which impacted model performance in negative ways.

2. It is super important to have a validation set. A validation helps to tune hyperparameters during the training process. By having a validation set, we were able to improve the performance by almost 20%, which is a great deal.

Overall, it may seem magical that neural networks are capable to learn such things, but once we explore the underlying mechanism, we will soon realize that there is no magic at all, and it all boils down to some concepts in calculus and linera algebra.

The next step would be to explore some language translation problems using the seq2seq model.



# Reference

- https://blog.keras.io/a-ten-minute-introduction-to-sequence-to-sequence-learning-in-keras.html
- https://keras.io/examples/nlp/lstm_seq2seq/
- http://colah.github.io/posts/2015-08-Understanding-LSTMs/
- https://distill.pub/2016/augmented-rnns/
- https://machinelearningmastery.com/learn-add-numbers-seq2seq-recurrent-neural-networks/